In [1]:
# итеративное чтение данных, удаление 5-го месяца, перевод форматов в более легкие
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import os
import gc

print("📥 Оптимизированная загрузка Parquet файла...")

# Путь к файлу
file_path = "/kaggle/input/sbolpro_merged_final.pqt"

# Сначала посмотрим на схему данных
table = pq.read_table(file_path, columns=['timestamp'])  # Только для анализа
print(f"Общее количество строк: {table.num_rows:,}")

# Метод 1: Загрузка с фильтрацией через PyArrow
def load_with_filter():
    """Загрузка с фильтрацией на уровне чтения"""
    print("\n🔍 Фильтрация на уровне чтения...")
    
    # Определяем границы дат
    import pyarrow.compute as pc
    
    # Читаем только колонку timestamp для фильтрации
    timestamps = pq.read_table(file_path, columns=['timestamp']).column('timestamp')
    
    # Преобразуем в datetime и фильтруем
    timestamps_dt = pd.to_datetime(timestamps.to_pandas())
    
    # Создаем маску для исключения мая 2023
    mask = ~((timestamps_dt.dt.year == 2023) & (timestamps_dt.dt.month == 5))
    
    print(f"Оставляем {mask.sum():,} строк из {len(mask):,}")
    
    # Загружаем данные с фильтрацией
    # Читаем файл по частям с фильтрацией
    batch_size = 100000
    filtered_chunks = []
    
    # Используем ParquetDataset для чтения с предикатами
    dataset = pq.ParquetFile(file_path)
    
    for i in range(0, table.num_rows, batch_size):
        # Читаем батч
        batch = dataset.read_row_group(i // dataset.metadata.num_row_groups if i > 0 else 0)
        batch_df = batch.to_pandas()
        
        # Применяем фильтр
        batch_df['timestamp'] = pd.to_datetime(batch_df['timestamp'])
        batch_df['year_month'] = batch_df['timestamp'].dt.strftime('%Y-%m')
        filtered = batch_df[batch_df['year_month'] != '2023-05']
        
        if len(filtered) > 0:
            filtered_chunks.append(filtered)
        
        # Очистка памяти
        if i % (batch_size * 10) == 0:
            gc.collect()
            print(f"Обработано {i:,} строк...")
    
    # Объединяем результаты
    result = pd.concat(filtered_chunks, ignore_index=True)
    print(f"✅ Загружено с фильтрацией: {len(result):,} строк")
    
    return result

# Метод 2: Загрузка только нужных колонок с оптимизацией типов
def load_optimized():
    """Загрузка с оптимизацией типов"""
    print("\n🎯 Загрузка с оптимизацией типов...")
    
    # Определяем типы данных для каждой колонки
    column_types = {
        'product_id': 'category',
        'user_id': 'category', 
        'is_sold': 'int8',
        # Остальные числовые колонки будем оптимизировать
    }
    
    # Сначала читаем небольшую выборку для анализа типов
    sample = pq.read_table(file_path).slice(0, 10000).to_pandas()
    
    # Анализируем числовые колонки
    for col in sample.select_dtypes(include=['float64']).columns:
        if col not in column_types:
            column_types[col] = 'float32'
    
    for col in sample.select_dtypes(include=['int64']).columns:
        if col not in column_types:
            min_val = sample[col].min()
            max_val = sample[col].max()
            if min_val >= 0:  # Беззнаковый
                if max_val < 256:
                    column_types[col] = 'uint8'
                elif max_val < 65536:
                    column_types[col] = 'uint16'
                elif max_val < 4294967296:
                    column_types[col] = 'uint32'
            else:  # Знаковый
                if min_val > -128 and max_val < 127:
                    column_types[col] = 'int8'
                elif min_val > -32768 and max_val < 32767:
                    column_types[col] = 'int16'
                elif min_val > -2147483648 and max_val < 2147483647:
                    column_types[col] = 'int32'
    
    print(f"Оптимизированные типы для {len(column_types)} колонок")
    
    # Загружаем с оптимизированными типами
    data = pd.read_parquet(
        file_path,
        columns=list(column_types.keys())  # Только нужные колонки
    )
    
    # Применяем оптимизированные типы
    for col, dtype in column_types.items():
        if col in data.columns:
            data[col] = data[col].astype(dtype)
    
    print(f"✅ Загружено: {len(data):,} строк, {data.memory_usage(deep=True).sum()/1024**2:.1f} MB")
    
    return data

# Метод 3: Итеративная обработка с немедленным удалением
def process_incremental():
    """Постепенная обработка с немедленным удалением ненужных данных"""
    print("\n🔄 Постепенная обработка...")
    
    # Читаем файл по частям
    batch_size = 50000
    result_chunks = []
    
    # Создаем ParquetFile объект
    pq_file = pq.ParquetFile(file_path)
    
    for i, batch in enumerate(pq_file.iter_batches(batch_size=batch_size)):
        df_batch = batch.to_pandas()
        
        # Немедленная фильтрация
        df_batch['timestamp'] = pd.to_datetime(df_batch['timestamp'])
        df_batch['year_month'] = df_batch['timestamp'].dt.strftime('%Y-%m')
        
        # Удаляем май 2023
        df_batch = df_batch[df_batch['year_month'] != '2023-05']
        
        if len(df_batch) > 0:
            # Оптимизируем типы
            for col in df_batch.select_dtypes(include=['float64']).columns:
                df_batch[col] = df_batch[col].astype('float32')
            
            result_chunks.append(df_batch)
        
        # Очистка памяти после каждого батча
        del df_batch
        gc.collect()
        
        if i % 20 == 0:
            print(f"Обработано {(i+1)*batch_size:,} строк...")
    
    # Объединяем результаты
    if result_chunks:
        result = pd.concat(result_chunks, ignore_index=True)
        print(f"✅ Итоговый размер: {len(result):,} строк")
        return result
    else:
        print("⚠️  Нет данных после фильтрации")
        return pd.DataFrame()

# Выбираем метод в зависимости от размера данных
try:
    # Пробуем самый эффективный метод
    data = process_incremental()
except MemoryError:
    print("Недостаточно памяти, пробуем более легкий метод...")
    data = load_optimized()
except Exception as e:
    print(f"Ошибка: {e}")
    print("Пробуем альтернативный метод...")
    data = load_optimized()

📥 Оптимизированная загрузка Parquet файла...
Общее количество строк: 1,056,889

🔄 Постепенная обработка...
Обработано 50,000 строк...
Обработано 1,050,000 строк...
✅ Итоговый размер: 1,051,557 строк


In [ ]:
# наши функции
def type_of_feature(df, threshold = 1000):
    num_cols = []
    cat_cols = []
    for col in df.columns:
        if df[col].nunique() < threshold:
            cat_cols.append(col)
        else:
            num_cols.append(col)
    return num_cols, cat_cols

# формируем train и test
import pandas as pd
import numpy as np


def target_constructor(df, id_product):
    """
    Разделяет данные для конкретного product_id на Train (прошлые месяцы) и OOT (последний месяц).
    """

    # Фильтруем датасет по нужному продукту
    df_prod = df[df['product_id'] == id_product].copy()

    # Проверка: если данных по продукту нет
    if df_prod.empty:
        print(f"Ошибка: Продукт {id_product} не найден в датасете.")
        return None, None, None, None

    # Убеждаемся, что timestamp — это дата
    df_prod['timestamp'] = pd.to_datetime(df_prod['timestamp'])

    # Создаем вспомогательную колонку 'period' (год-месяц) для удобного поиска последнего месяца
    df_prod['period'] = df_prod['timestamp'].dt.to_period('M')

    # Находим последний месяц (максимальный период)
    last_period = df_prod['period'].max()

    # Разделяем на Train (всё кроме последнего) и OOT (только последний)
    mask_oot = df_prod['period'] == last_period

    df_train_full = df_prod[~mask_oot]  # Все прошлые месяцы
    df_oot_full = df_prod[mask_oot]  # Последний месяц

    # Формируем X и y
    # X - удаляем таргет (и вспомогательную колонку period)
    # y - только таргет
    x_train = df_train_full.drop(columns=['is_sold', 'period'])
    y_train = df_train_full['is_sold']

    x_oot = df_oot_full.drop(columns=['is_sold', 'period'])
    y_oot = df_oot_full['is_sold']

    # --- БЛОК ВЫВОДА ИНФОРМАЦИИ (PRINT) ---
    print("=" * 60)
    print(f"СТАТИСТИКА ПО ПРОДУКТУ: {id_product}")
    print("=" * 60)

    # Размеры выборок
    print(f"Размер x_train: {x_train.shape[0]} строк")
    print(f"Размер y_train: {y_train.shape[0]} строк")
    print(f"Размер x_oot:   {x_oot.shape[0]} строк")
    print(f"Размер y_oot:   {y_oot.shape[0]} строк")
    print("-" * 30)

    # Информация о периодах
    # Берем последний месяц из трейна (если трейн не пустой)
    if not df_train_full.empty:
        max_train_date = df_train_full['timestamp'].max()
        print(f"Последний месяц в TRAIN: {max_train_date.strftime('%B %Y')}")
    else:
        print("TRAIN пуст (возможно, у продукта данные только за 1 месяц)")

    # Месяц OOT
    print(f"Месяц OOT (Test):        {last_period.strftime('%B %Y')}")
    print("-" * 30)

    # Статистика по is_sold (конверсия) по месяцам
    print("Распределение is_sold (процент продаж) по месяцам:")

    # Объединяем обратно временно для красивого группирования, или используем исходный df_prod
    stats = df_prod.groupby('period')['is_sold'].agg(['count', 'mean'])
    stats['mean'] = (stats['mean'] * 100).round(2)
    stats.columns = ['Кол-во записей', 'Процент продаж %']

    # Помечаем, какая строка попала в OOT
    stats['Тип выборки'] = stats.index.map(lambda x: 'OOT' if x == last_period else 'TRAIN')

    print(stats)
    print("=" * 60 + "\n")

    return x_train, y_train, x_oot, y_oot


import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_auc_score, 
    average_precision_score,
    roc_curve,
    precision_recall_curve
)

def metric_report(y_true, y_pred, ks=(1, 3, 5, 10, 20)):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    roc_auc = roc_auc_score(y_true, y_pred)
    pr_auc = average_precision_score(y_true, y_pred)

    order = np.argsort(-y_pred)
    y_true_sorted = y_true[order]

    n = len(y_true)
    total_pos = y_true.sum()

    precision_at_k = {}
    recall_at_k = {}

    for k in ks:
        if k <= 1:
            top_k = k
        else:
            top_k = int(n * (k / 100))

        top_k = max(1, top_k)
        y_top = y_true_sorted[:top_k]

        if k <= 10:
            precision_at_k[str(k)] = y_top.mean()

        if k <= 20:
            recall_at_k[str(k)] = (
                y_top.sum() / total_pos if total_pos > 0 else 0
            )

    fpr, tpr, _ = roc_curve(y_true, y_pred)

    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.4f}", lw=2)
    plt.plot([0, 1], [0, 1], "k--", lw=1)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.grid(True)
    plt.legend(loc="lower right")
    plt.show()

    precision, recall, _ = precision_recall_curve(y_true, y_pred)

    plt.figure(figsize=(6, 5))
    plt.plot(recall, precision, lw=2, label=f"PR-AUC = {pr_auc:.4f}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve")
    plt.grid(True)
    plt.legend(loc="lower left")
    plt.show()

    return {
        "roc_auc": roc_auc,
        "pr_auc": pr_auc,
        "precision@k": precision_at_k,
        "recall@k": recall_at_k
    }

In [ ]:
products = [1, 6, 2]

In [ ]:
# перебираем threshold в cat_cols
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


for threshold in [10, 15, 20, 50, 100]:
    num_cols, cat_cols = type_of_feature(data, threshold)
    if 'timestamp' in cat_cols:
        cat_cols.remove('timestamp')
    if 'is_sold' in cat_cols:
        cat_cols.remove('is_sold')
    if 'product_id' in cat_cols:
        cat_cols.remove('product_id')
    if 'year_month' in cat_cols:
        cat_cols.remove('year_month')
    print(f'Число категориальных признаков: {len(cat_cols)}')
    for product in products:
        x_train, y_train, x_oot, y_oot = target_constructor(data, product)
        for col in cat_cols:
            if col in x_train.columns:
                x_train[col] = x_train[col].astype(str)
            if col in x_oot.columns:
                x_oot[col] = x_oot[col].astype(str)
        model = CatBoostClassifier(
        iterations=100,           # Уменьшил для быстрого теста
        learning_rate=0.05,
        depth=4,                  # Меньшая глубина для избежания переобучения
        # l2_leaf_reg=10,           # Регуляризация
        auto_class_weights='Balanced',      # Автоматические веса
        eval_metric='F1',
        random_seed=42,
        cat_features=cat_cols
    )
        print('обучение')
        # Обучаем модель
        model.fit(
        x_train.drop(['timestamp', 'user_id', 'product_id'], axis=1), 
        y_train,
        early_stopping_rounds=50,
        verbose=10 # logs
    )
        print("\n📊 Оценка модели:")
        
        # Предсказания
        y_pred = model.predict(x_oot.drop(['timestamp', 'user_id', 'product_id'], axis=1))
        y_pred_proba = model.predict_proba(x_oot.drop(['timestamp', 'user_id', 'product_id'], axis=1))[:, 1]
        print('sum(y_pred), len(y_pred), sum(y_pred) / len(y_pred)')
        print(sum(y_pred), len(y_pred), sum(y_pred) / len(y_pred))
        print(f"Accuracy:  {accuracy_score(y_oot, y_pred):.4f}")
        print(f"Precision: {precision_score(y_oot, y_pred, zero_division=0):.4f}")
        print(f"Recall:    {recall_score(y_oot, y_pred, zero_division=0):.4f}")
        print(f"F1-Score:  {f1_score(y_oot, y_pred, zero_division=0):.4f}")
        print(f"ROC-AUC:   {roc_auc_score(y_oot, y_pred_proba):.4f}")
        metric_report(y_oot, y_pred_proba, ks=(1, 3, 5, 10, 20))